In [1]:
import subprocess
import pdb

import os
import glob
import numpy as np
from numpy.fft import fft, ifft, fftfreq
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.animation import FuncAnimation
from nilearn.plotting import view_img_on_surf, view_img
from scipy.stats import ttest_1samp

import networkx as nx
from IPython.display import HTML

# import nest_asyncio
# nest_asyncio.apply()
# import datalad.api as dl

import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")

import pandas as pd
import scipy.io
from scipy import stats

from sklearn.manifold import MDS
import scipy.spatial.distance as sp_distance
from sklearn.preprocessing import RobustScaler, StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns 
from mpl_toolkits.mplot3d import Axes3D

import time
from copy import deepcopy
import numpy as np
import pandas as pd 

from nilearn import datasets
from nilearn import surface
from nilearn import plotting
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
import nibabel as nib

from nltools.data import Brain_Data, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import isc, isfc, isps, fdr, threshold, phase_randomize, circle_shift, _butter_bandpass_filter, _phase_mean_angle, _phase_vector_length
from nilearn.plotting import view_img_on_surf, view_img
from sklearn.metrics import pairwise_distances
from sklearn.utils import check_random_state

# from brainiak import image, io
# from brainiak.isc import isc, isfc, permutation_isc
import matplotlib.pyplot as plt
import seaborn as sns 

from datetime import datetime as dt
from scipy.signal import hilbert, butter, filtfilt
from kuramoto import Kuramoto, plot_phase_coherence, plot_activity

%autosave 5
%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

# %matplotlib notebook
%matplotlib inline
%autosave 5
sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})
sns.set(palette="colorblind")

from py3plex.core import multinet
from py3plex.core import random_generators
from py3plex.algorithms.community_detection import infomap
import numpy as np
import queue
import matplotlib.pyplot as plt
import seaborn as sns

from ts2vg import NaturalVG
from ts2vg import HorizontalVG
import pickle
from datetime import datetime
from time import time
from scipy import stats

from py3plex.core.parsers import  parse_network
from py3plex.core import multinet

import numpy as np
import pylab

from tqdm import tqdm

from multiprocess import Pool
from py3plex.algorithms.statistics.basic_statistics import core_network_statistics, identify_n_hubs

from py3plex.algorithms.community_detection import community_wrapper as cw
from py3plex.core import multinet
from py3plex.visualization.multilayer import hairball_plot, plt
from py3plex.visualization.colors import colors_default
from collections import Counter

import community as community_louvain
from py3plex.algorithms.multilayer_algorithms.entanglement import compute_entanglement_analysis



Autosaving every 5 seconds


Autosaving every 5 seconds
3.9.12 (main, Jun  1 2022, 06:36:29) 
[Clang 12.0.0 ]


# Functions to register the images
Normal registrations lead to severe stretching an distortion of the functional images. Therefore, the ROIs (in MNI asym 09) and the T1 were registered to the functional image. While not standard, because this is an ROI analysis not whole brain we feel that this method is the best way to get the true areas of the ROIs in the functional images

In [ ]:
def get_t1(strc, basepath):
    T1 = strc
    sub = strc.split('/')[-1].split('.')[0].split('_')[-1]
    output_path = os.path.join(basepath,'derivatives','reg')
    return([sub,output_path,T1])

In [ ]:
def get_example_func(reference,T1,output_path,sub,style):
    example_func = '%s/%s_%s_example_func.nii.gz'%(output_path,sub,style)
    example_func_cmd = '/usr/local/fsl/bin/fslroi %s %s 300 1'%(reference,example_func)
    example_func_cmd_list= example_func_cmd.split(' ')
    subprocess.call(example_func_cmd_list)
    print(example_func)
    return(example_func)

In [ ]:
def func_thresh(example_func,output_path,sub):
    # threshold out extra bullshit
    print('thresholding')
    x = example_func.split('.')[0]
    x = '%s_thr'%x
    print(x)
    thr_cmd = '/usr/local/fsl/bin/fslmaths %s -thr 200 %s'%(example_func,x)
    thr_cmd_list = thr_cmd.split(' ')
    subprocess.call(thr_cmd_list)
    #example_func_thr = '%s/%s_example_func_thr.nii.gz'%(output_path,sub)
    example_func_thr = '%s.nii.gz'%x
    return(example_func_thr)

In [ ]:
def heavy_lifting(T1,example_func_thr, output_path, sub, basepath):
# register the T1 to the BOLD
    ## get the matrix
    print('welcome to the matrix')
    x = example_func.split('.')[0]
    outp = '%s_T12func.mat'%x
    cmd = 'flirt -in %s -ref %s -out outputvol -omat %s'%(T1,example_func_thr,outp)
    cmd_list = cmd.split(' ')
    subprocess.call(cmd_list)
    ## use the matrix
    print('neo use the matrix')
    matrix = outp
    y = '%s_T12func'%x
    flirt_cmd = 'flirt -in %s -ref %s -applyxfm -init %s -out %s'%(T1, example_func_thr,matrix,y)
    flrt_cmd_list = flirt_cmd.split(' ')
    subprocess.call(flrt_cmd_list)
    # Register the atlas image to the T1
    print('registering')
    T1_reg = y
    hi2std = '%s_highres2standard'%x
    hi2stdmat = '%s_highres2standard.mat'%x
    standard = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
    s2t1_cmd = '/usr/local/fsl/bin/flirt -in %s -ref %s -out %s -omat %s -cost corratio -dof 12 -searchrx -90 90 -searchry -90 90 -searchrz -90 90 -interp trilinear'%(standard,T1_reg, hi2std,hi2stdmat)
    s2t1_cmd_list = s2t1_cmd.split(' ')
    subprocess.call(s2t1_cmd_list)
    return(hi2stdmat, T1_reg)

In [ ]:
def roi_dir(mask_dir, sub, style):
    roi_out = os.path.join(mask_dir,'%s_%s'%(style,sub))
    try:
        os.makedirs(roi_out)
    except FileExistsError:
        pass
    return(roi_out)


In [ ]:
def roi_create(roi_orig, roi_out, T1_reg, matrix):
    roi = roi_orig.split('/')[-1].split('_')[-1].split('.')[0]
    print(roi)
    ### align with the standard brain
    outp = '%s/%s_roi2T1'%(roi_out,roi)
    roi_cmd = 'flirt -in %s -ref %s -applyxfm -init %s -out %s'%(roi_orig,T1_reg,matrix,outp)
    print(roi_cmd)
    roi_cmd_list = roi_cmd.split(' ')
    subprocess.call(roi_cmd_list)
    return(outp, roi)

In [ ]:
def roi_thresh(roi,roi_out,thresh):
    inp ='%s/%s_roi2T1.nii.gz'%(roi_out,roi)
    oot = '%s/%s_%s_roi2T1_thresh'%(roi_out,roi,thresh)
    print(thresh)
    thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
    print(thresh_cmd)
    thresh_cmd_list = thresh_cmd.split(' ')
    subprocess.run(thresh_cmd_list)
    num = roi.split('l')[1]
    oupt = '%s/%s_%s_roi2T1_thresh_bin'%(roi_out,roi,thresh)
    bin_cmd = 'fslmaths %s -bin -mul %s %s'%(oot,num,oupt)
    bin_cmd_list = bin_cmd.split(' ')
    subprocess.run(bin_cmd_list)
    return(oupt)

In [3]:
def onetoughjar(path2dic):
    p = os.path.join(path2dic)
    list_of_files = glob.glob(p) # * means all if need specific format then *.csv
    print(list_of_files)
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    with open(latest_file, 'rb') as pickle_file:
        try:
            while True:
                output = pickle.load(pickle_file)
        except EOFError:
            pass
    return(output)
    
    
    
    
    

In [4]:
def adillyofapickle(basepath, dic, name):
    datefmt='%m-%d-%Y_%I-%M-%S'
    st = datetime.fromtimestamp(time()).strftime(datefmt)
    if os.path.exists(os.path.join(basepath,'tmp')):
        print('already have tmp')
    else:
        os.makedirs(os.path.join(basepath,'tmp'))
    pickle.dump(dic, open(os.path.join(basepath,'tmp','%s_%s'%(name,st)), 'wb'), protocol=4)


In [ ]:
def extractor(nifti, output):
    bet_cmd = 'bet %s %s -F'%(nifti, output)
    subprocess.run(['bet','%s'%nifti,'%s'%output,'-F'])

In [ ]:
basepath = '/Users/gracer/Documents/BBC_21-2939_prepro_data'

In [31]:
tasks = ['base','clamp']

## Need to register the ROIS from neurosynth to the same template as the others

In [ ]:
inp = os.path.join(basepath,'atlases','anatomical.nii.gz')
out = os.path.join(basepath,'atlases','anatomical_asym.nii.gz')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
outmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl_cmd = 'flirt -in %s -ref %s -out %s -omat %s'%(inp, ref, out,outmat)
fl_cmd_list = fl_cmd.split(' ')
subprocess.run(fl_cmd_list)


In [ ]:
inp = os.path.join(basepath,'atlases','dlpfc_association-test_z_FDR_0.01.nii.gz')
out = os.path.join(basepath,'atlases','atlas','CIT168toMNI152_prob_atlas_bilat_1mm__vol17')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
inmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl2_cmd = 'flirt -in %s -ref %s -out %s -init %s -applyxfm'%(inp, ref, out, inmat)
fl2_cmd_list = fl2_cmd.split(' ')
subprocess.run(fl2_cmd_list)


In [ ]:
inp = os.path.join(basepath,'atlases','visual_cortex_association-test_z_FDR_0.01.nii.gz')
out = os.path.join(basepath,'atlases','atlas','CIT168toMNI152_prob_atlas_bilat_1mm__vol18')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
inmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl3_cmd = 'flirt -in %s -ref %s -out %s -init %s -applyxfm'%(inp, ref, out, inmat)
fl3_cmd_list = fl3_cmd.split(' ')
subprocess.run(fl3_cmd_list)


In [ ]:
inp = os.path.join(basepath,'atlases','hippocampus_association-test_z_FDR_0.01.nii.gz')
out = os.path.join(basepath,'atlases','atlas','CIT168toMNI152_prob_atlas_bilat_1mm__vol19')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
inmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl4_cmd = 'flirt -in %s -ref %s -out %s -init %s -applyxfm'%(inp, ref, out, inmat)
fl4_cmd_list = fl4_cmd.split(' ')
subprocess.run(fl4_cmd_list)

In [ ]:
inp = os.path.join(basepath,'atlases','anterior_cingulate_association-test_z_FDR_0.01.nii.gz')
out = os.path.join(basepath,'atlases','atlas','CIT168toMNI152_prob_atlas_bilat_1mm__vol20')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
inmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl5_cmd = 'flirt -in %s -ref %s -out %s -init %s -applyxfm'%(inp, ref, out, inmat)
fl5_cmd_list = fl5_cmd.split(' ')
subprocess.run(fl5_cmd_list)

In [ ]:
inp = os.path.join(basepath,'atlases','insula_association-test_z_FDR_0.01.nii.gz')
out = os.path.join(basepath,'atlases','atlas','CIT168toMNI152_prob_atlas_bilat_1mm__vol21')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
inmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl6_cmd = 'flirt -in %s -ref %s -out %s -init %s -applyxfm'%(inp, ref, out, inmat)
fl6_cmd_list = fl6_cmd.split(' ')
subprocess.run(fl6_cmd_list)

In [ ]:
inp = os.path.join(basepath,'atlases','middle_temporal_association-test_z_FDR_0.01.nii.gz')
out = os.path.join(basepath,'atlases','atlas','CIT168toMNI152_prob_atlas_bilat_1mm__vol22')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
inmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl7_cmd = 'flirt -in %s -ref %s -out %s -init %s -applyxfm'%(inp, ref, out, inmat)
fl7_cmd_list = fl7_cmd.split(' ')
subprocess.run(fl7_cmd_list)

In [ ]:
inp = os.path.join(basepath,'atlases','posterior_cingulate_association-test_z_FDR_0.01.nii.gz')
out = os.path.join(basepath,'atlases','atlas','CIT168toMNI152_prob_atlas_bilat_1mm__vol23')
ref = os.path.join(basepath,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
inmat = os.path.join(basepath,'atlases','anatomical_asym.mat')
fl8_cmd = 'flirt -in %s -ref %s -out %s -init %s -applyxfm'%(inp, ref, out, inmat)
fl8_cmd_list = fl8_cmd.split(' ')
subprocess.run(fl8_cmd_list)

## Needed to prethreshold these. They are from neurosyth and pick up a lot of excess areas

In [ ]:
inp = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/CIT168toMNI152_prob_atlas_bilat_1mm__vol19.nii.gz'
oot = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/threshCIT168toMNI152_prob_atlas_bilat_1mm__vol19.nii.gz'
thresh = '11'
thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
print(thresh_cmd)
thresh_cmd_list = thresh_cmd.split(' ')
subprocess.run(thresh_cmd_list)

In [ ]:
inp = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/CIT168toMNI152_prob_atlas_bilat_1mm__vol18.nii.gz'
oot = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/threshCIT168toMNI152_prob_atlas_bilat_1mm__vol18.nii.gz'
thresh = '3'
thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
print(thresh_cmd)
thresh_cmd_list = thresh_cmd.split(' ')
subprocess.run(thresh_cmd_list)

In [ ]:
inp = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/CIT168toMNI152_prob_atlas_bilat_1mm__vol17.nii.gz'
oot = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/threshCIT168toMNI152_prob_atlas_bilat_1mm__vol17.nii.gz'
thresh = '6'
thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
print(thresh_cmd)
thresh_cmd_list = thresh_cmd.split(' ')
subprocess.run(thresh_cmd_list)

In [ ]:
inp = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/CIT168toMNI152_prob_atlas_bilat_1mm__vol20.nii.gz'
oot = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/threshCIT168toMNI152_prob_atlas_bilat_1mm__vol20.nii.gz'
thresh = '8'
thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
print(thresh_cmd)
thresh_cmd_list = thresh_cmd.split(' ')
subprocess.run(thresh_cmd_list)

In [ ]:
inp = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/CIT168toMNI152_prob_atlas_bilat_1mm__vol21.nii.gz'
oot = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/threshCIT168toMNI152_prob_atlas_bilat_1mm__vol21.nii.gz'
thresh = '9'
thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
print(thresh_cmd)
thresh_cmd_list = thresh_cmd.split(' ')
subprocess.run(thresh_cmd_list)

In [ ]:
inp = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/CIT168toMNI152_prob_atlas_bilat_1mm__vol22.nii.gz'
oot = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/threshCIT168toMNI152_prob_atlas_bilat_1mm__vol22.nii.gz'
thresh = '8'
thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
print(thresh_cmd)
thresh_cmd_list = thresh_cmd.split(' ')
subprocess.run(thresh_cmd_list)

In [ ]:
inp = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/CIT168toMNI152_prob_atlas_bilat_1mm__vol23.nii.gz'
oot = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas/threshCIT168toMNI152_prob_atlas_bilat_1mm__vol23.nii.gz'
thresh = '9'
thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
print(thresh_cmd)
thresh_cmd_list = thresh_cmd.split(' ')
subprocess.run(thresh_cmd_list)




# Skull stripping

In [ ]:
for task in tasks:
    print(task)
    for nifti in glob.glob(os.path.join(basepath,'func','%s'%task,'*.nii')):
        print(nifti)
        name = nifti.split('/')[-1].split('.')[0]
        output = os.path.join(basepath,'derivatives','BET','%s_brain'%name)
        extractor(nifti, output)

# Registration

## Try not to run unless you got time

In [ ]:
for strc in glob.glob(os.path.join(basepath, 'anat','ss_t1*.nii')):
    sub,output_path,T1 = get_t1(strc,basepath)
    print(sub)
    for task in tasks:
        for ref in glob.glob(os.path.join(basepath,'derivatives','BET','%s'%task,'*%s*_brain.nii.gz'%sub)):
            style = ref.split('_')[4]
            example_func = get_example_func(ref,T1,output_path,sub,style)
            example_func_thr = func_thresh(example_func,output_path,sub)
            matrix, T1_reg = heavy_lifting(T1, example_func_thr, output_path, sub, basepath)
            mask_dir = os.path.join(basepath,'atlases','atlas')
            for x in glob.glob(os.path.join(mask_dir,'CIT168toMNI152_prob_atlas_bilat_1mm__vol*.nii.gz')):
                
                roi_orig = os.path.join(mask_dir,x)
                roi_out = roi_dir(mask_dir, sub,style)
                roi_path,roi = roi_create(roi_orig, roi_out, T1_reg, matrix)
                roithr = roi_thresh(roi,roi_out,'0.5')


Exlcuding the PBP, VTA, SNr due to low signal. Need to fix roi_merger so it can predict the number of inputs

In [ ]:
thresh = 0.5
for path2rois in glob.glob(os.path.join('/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas','*_*/')):
    print(path2rois)
    rois = glob.glob(os.path.join(path2rois,'vol*_%s_roi2T1_thresh_bin.nii.gz'%thresh))
   

In [ ]:
def roi_merger(path2rois, thresh):
    ## make a total brain
    rois = glob.glob(os.path.join(path2rois,'vol*_%s_roi2T1_thresh_bin.nii.gz'%thresh))
    print(len(rois))
    text = '-add %s '
    repeated = text * len(rois[1:])
    finalgirl = os.path.join(path2rois,'multilayer_rois.nii.gz')
    rois.append(finalgirl)    
    cmd = 'fslmaths %s '+repeated+'%s'
    roi_cmd2 = cmd%tuple(rois)
    roi_cmd2_list=roi_cmd2.split(' ')
    print(roi_cmd2_list)
    subprocess.run(roi_cmd2_list)
    

In [ ]:
for path2rois in glob.glob(os.path.join('/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas','*_*/')):
#     print(path2rois)
    roi_merger(path2rois, '0.5')

# Parcellation 

In [ ]:
def parcellator(basepath, func, corrlabel):
    print("starting parcellation")
    corrtxt = os.path.join(basepath,'multilayer_corrlabel_ts.txt')
    cmd = 'fslmeants -i %s --label=%s -o %s'%(func, corrlabel, corrtxt)
    cmd_list = cmd.split(' ')
    print(cmd_list)
    subprocess.run(cmd_list)

In [ ]:
outpath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output'
for task in tasks:
    for func in glob.glob(os.path.join(basepath,'derivatives','BET','%s'%task,'*_brain.nii.gz')):
        sub = func.split('/')[-1].split('_')[0].split('u')[-1]
        style = func.split('/')[-1].split('_')[1]
        print(sub)
        print(style)
        try:
            op = os.path.join(outpath,style,sub)
            os.makedirs(op)
        except FileExistsError:
            pass
        corrlabel = os.path.join('/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas','%s_%s'%(style,sub),'multilayer_rois.nii.gz')
        parcellator(op, func, corrlabel)


# Insulin kinetics

In [33]:
#!/usr/bin/env python
# Implementation of algorithm from https://stackoverflow.com/a/22640362/6029703

def thresholding_algo(y, lag, threshold, influence):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.median(y[0:lag])
    stdFilter[lag - 1] = stats.median_absolute_deviation(y[0:lag])
    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
def ins_convert(insulin_units, mol):
    pmol = np.multiply(insulin_units, 6.0) 
    if mol == 'mol':
        mole = 1000
        mol = np.divide(pmol, mole)
        return(mol)
    else:
        return(mol)

In [ ]:
path2insulin = os.path.join('/Users/gracer/Documents/BBC_21-2939_insulin','*.csv')
converted_ins = []
for sub in glob.glob(path2insulin):
    print(sub)
    df = pd.read_csv(sub, sep = ',', encoding='latin-1')
    converted_ins.append(df)
    
    
ins_conv_df = pd.concat(converted_ins)

In [ ]:
ins_conv_df

In [ ]:
pd.Series(ins_conv_df['pid']).unique()

## Creating a boxcar function from the standardized insulin timeseries

In [ ]:
z_ins = []
subs = []
for x in list(pd.Series(ins_conv_df['pid']).unique()):
    ins = np.array(ins_conv_df.loc[ins_conv_df['pid'] == x]['insulin_uIU_mL']).reshape(-1, 1)
    print(x)
    inscaler = StandardScaler()
    X = pd.DataFrame(inscaler.fit_transform(ins))
    X['sub'] = x
    z_ins.append(X)

In [ ]:
z_ins = pd.concat(z_ins)

In [ ]:
lag = 180
threshold = 1
influence = 1
thresh_alg = {}
for x in list(pd.Series(ins_conv_df['pid']).unique()):
    y = np.array(z_ins.loc[z_ins['sub'] == x][0].repeat(60))
    z  = thresholding_algo(y, lag, threshold, influence)
    thresh_alg['%s'%x]=z


In [ ]:
thresh_alg['20001']['signals']

In [ ]:
plt.figure(figsize=(16, 8), dpi=150)
pd.DataFrame(x['signals']).plot(label='%s phase 20004', color = 'violet')
pd.DataFrame(y).plot(label='%s phase 20004', color = 'blue')
pd.DataFrame(z_ins.loc[z_ins['sub'] == 20010][0]).plot(label='%s phase 20004', color = 'blue')

# adding title to the plot
plt.title('ROI phase')

# adding Label to the x-axis
plt.xlabel('time')

# adding legend to the curve
plt.legend()

# Standardizing the ROI timeseries 

In [26]:
def combo(path2ts, outpath, instatus, model):
    prets = pd.read_csv(path2ts, sep = '  ', header = None)
    prets = prets.iloc[:, : len(heads)]
    
    transformer = StandardScaler().fit(prets)
    ts = pd.DataFrame(data = transformer.transform(prets))

    ts = ts.set_axis(heads, axis=1, inplace=False)
    ts['TR'] = ts.index
    ts['TR'] = pd.to_datetime(ts['TR'], unit='s')
#     pdb.set_trace()
    if instatus == 'clamp':
        p = ts.shape[0]-len(model)
        model = np.pad(model, (0, p), 'constant', constant_values=(0, 0))
        ts['model'] = model
    ts.to_csv(outpath, index=False)

In [27]:
labels = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/CIT168_Reinf_Learn_v1.1.0/labels.csv'
bbasepath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base'
clamppath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/clamp'
ppath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/'

In [28]:
ROI_names = pd.read_csv(labels,sep = ',')
heads = list(ROI_names['ROI'])

In [29]:
len(heads)

23

In [34]:
for task in tasks:
    for sub_path in glob.glob(os.path.join(ppath,'%s'%task,'*')):
        print(sub_path)
        sub = sub_path.split('/')[-1]
        style = sub_path.split('/')[-2]
        path2ts = os.path.join(sub_path,'multilayer_corrlabel_ts.txt')
        model = thresh_alg['%s'%sub]['signals']
        outpath = os.path.join(sub_path,'multilayer_total_ts.csv')
        combo(path2ts, outpath, style, model)

/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base/20008


NameError: name 'thresh_alg' is not defined

In [ ]:
len(thresh_alg['20001']['signals'])

In [ ]:
multilayer_total_ts = pd.read_csv('/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/clamp/20001/multilayer_total_ts.csv',sep = ',')


In [ ]:
plt.figure(figsize=(16, 8), dpi=150)
multilayer_total_ts['Pu'].plot(label='Pu', color = 'violet')
multilayer_total_ts['model'].plot(label='model', color = 'blue')

# adding title to the plot
plt.title('ROI phase')

# adding Label to the x-axis
plt.xlabel('time')

# adding legend to the curve
plt.legend()

# Time series to visible graph

In [110]:
datapath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/clamp'

In [125]:
model_pattern = {}
for item in glob.glob(os.path.join(datapath,'*','multilayer_total_ts.csv')):
    sub = item.split('/')[-2]
    all_ts = pd.read_csv(item)
    all_ts['diff'] = all_ts['model'].diff()
    df_filtered = all_ts[all_ts['diff'] != 0]
    model_pattern[sub] = df_filtered[['model','diff']]
    

In [124]:
df_filtered

,Pu,Ca,NAC,EXA,GPe,GPi,SNc,RN,SNr,PBP,...,dlPFC,Viz,Hippo,AC,Ins,MTG,PC,TR,model,diff
0,0.964552,0.191570,3.807908,1.082435,3.069511,3.691943,3.206632,2.358542,0.0,0.0,...,1.376586,0.733798,1.431294,0.509393,-1.208204,2.790418,2.289917,1970-01-01 00:00:00,0.0,NaN
180,-0.763206,-0.364052,0.858805,-0.884686,0.118457,0.847592,0.485769,-0.041368,0.0,0.0,...,-0.078777,-0.330119,0.209219,-0.747960,-1.855674,0.201805,1.104005,1970-01-01 00:03:00,1.0,1.0
208,-0.263048,-0.089901,0.955777,-0.638796,0.840618,1.187826,0.226640,-0.009370,0.0,0.0,...,-0.066903,-0.687671,1.087259,-0.502852,-1.672859,0.418514,-0.680809,1970-01-01 00:03:28,0.0,-1.0
274,-0.983115,-0.909099,-0.179371,-2.319045,-0.639367,0.065056,0.097075,-0.798673,0.0,0.0,...,-0.479086,-1.674337,-0.258402,-0.792228,-1.395442,-0.073045,-0.352578,1970-01-01 00:04:34,-1.0,-1.0
300,-0.688786,-0.530820,-0.133737,-0.761741,0.421586,0.786350,0.356205,0.422614,0.0,0.0,...,-0.524884,-0.168958,0.422988,-0.861027,-1.740923,-0.795848,0.069841,1970-01-01 00:05:00,0.0,1.0
360,-0.563411,0.522666,0.311196,-0.126525,1.045676,1.419184,0.097075,-0.062701,0.0,0.0,...,-0.650405,-2.081541,-0.231681,-0.031991,-1.394951,-0.674280,-0.413943,1970-01-01 00:06:00,-1.0,-1.0
480,-1.005240,-0.901777,0.140068,-1.888737,-0.692860,-0.663043,-0.680315,-0.649346,0.0,0.0,...,-1.944626,-1.727317,-0.841257,-0.334591,-1.170118,-0.822276,-0.175619,1970-01-01 00:08:00,0.0,1.0
540,-0.482956,-0.109426,1.674514,-0.433888,1.429046,1.262677,0.744899,0.598608,0.0,0.0,...,-0.250096,-0.228734,0.728611,-0.099832,-1.027355,1.669875,1.203426,1970-01-01 00:09:00,1.0,1.0
660,-1.842636,-1.079934,0.094434,-1.396957,-0.782016,-0.397661,-0.421185,-0.382689,0.0,0.0,...,-1.138918,-1.152294,-0.444615,-0.993642,-1.942167,0.315445,-0.643704,1970-01-01 00:11:00,-1.0,-2.0
720,-0.432002,0.226551,-0.019651,-0.966650,-0.483344,-0.037014,0.874464,0.939928,0.0,0.0,...,-1.254262,-0.328871,0.659721,-0.256594,-0.863706,-1.029735,-0.079528,1970-01-01 00:12:00,0.0,1.0


In [126]:
model_pattern['20001']

,model,diff
0,0.0,NaN
180,1.0,1.0
249,0.0,-1.0
300,-1.0,-1.0
420,1.0,2.0
559,0.0,-1.0
600,-1.0,-1.0
660,0.0,1.0
720,1.0,1.0
780,0.0,-1.0


In [115]:
model_pattern

{'20008': [0.0, -1.0, 1.0],
 '20001': [0.0, 1.0, -1.0],
 '20002': [0.0, -1.0, 1.0],
 '20003': [0.0, 1.0, -1.0],
 '20004': [0.0, -1.0, 1.0],
 '20010': [0.0, 1.0, -1.0]}

In [ ]:
mdels = {'high':{}, 'neu':{},'low':{}}
for name in mdels.keys():
    for subs in glob.glob(os.path.join(datapath,'*','multilayer_total_ts.csv')):
            sub = subs.split('/')[-2]
            all_ts = pd.read_csv(subs)
            mdels[name][sub] = {}
            for roi in list(all_ts.columns[0:-2]):
                mdels[name][sub][roi] = []
                all_ts['diff'] = all_ts['model'].diff()
                df_filtered = all_ts[all_ts['diff'] != 0]
                for i, ind in enumerate(list(df_filtered.index)):
                    try:
                        a = list(df_filtered.index)[i]
                        b = list(df_filtered.index)[i+1]
                        c = df_filtered['model'].loc[a:b-1].mean()

                        if name == 'high' and c == 1:
                            mdels[name][sub][roi].append(all_ts[roi].loc[a:b])
                        if name == 'neu' and c == 0:
                            mdels[name][sub][roi].append(all_ts[roi].loc[a:b])
                        if name == 'low' and c == -1:
                            mdels[name][sub][roi].append(all_ts[roi].loc[a:b])
                    except IndexError:
                        print('end of list')


In [ ]:
adillyofapickle(savepath,mdels,'mdels')

In [52]:
mdels = onetoughjar(os.path.join(savepath,'tmp','mdels*'))

['/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/mdels_06-30-2022_01-24-43']
/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/mdels_06-30-2022_01-24-43


KeyboardInterrupt: 

In [ ]:
def vizgraph(ts0):
    mapping = {}
    for i, x in enumerate(np.array(ts0.index)):
        mapping.update({i:x})
    ts = np.array(ts0)
    nat = NaturalVG(weighted = 'distance').build(ts, xs=np.array(ts0.index))
    nat_mat = nat.adjacency_matrix(use_weights=True, no_weight_value = 0)
    nat_matz = stats.zscore(np.array(nat_mat))
    natG = nx.from_numpy_matrix(nat_matz)
    return({'natural_nx':natG, 'natural':nat,'node_maps':mapping, 'ts':ts, 'nat_matz':nat_matz})



In [ ]:
graphs = {'high':{}, 'neu':{},'low':{}}
for key, data in mdels.items():
    for sub, rois in data.items():
        graphs[key][sub] ={}
        for roi, list_ts in rois.items():
            graphs[key][sub][roi]= {}
            for i, ts in enumerate(list_ts):
                graphs[key][sub][roi][i]= vizgraph(ts)

In [ ]:
adillyofapickle(savepath,graphs,'graphs_weight_dist_smz')

In [36]:
X = onetoughjar(os.path.join(savepath,'tmp','graphs_weight_dist_smz*'))

['/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/graphs_weight_dist_smz_06-23-2022_05-03-32', '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/graphs_weight_dist_smz_06-23-2022_05-01-20', '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/graphs_weight_dist_smz_06-30-2022_01-27-24']
/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/graphs_weight_dist_smz_06-30-2022_01-27-24


In [ ]:
delete = ['GPe', 'GPi', 'SNc', 'RN', 'SNr', 'PBP', 'VTA', 'VeP', 'HN', 'HTH', 'MN', 'STH','Viz','AC', 'MTG', 'PC']

In [ ]:
for level, data in graphs.items():
    for sub, dat in data.items():
        for z in delete:
            del dat[z]

In [ ]:
adillyofapickle(savepath,graphs,'graphs_weight_dist_abbr_smz')

## Creating inter-brain correlations overtime

In [ ]:
inter_roi = {}
for key, data in graphs.items():
    inter_roi[key] = {}
    for sub, dat in data.items():
        inter_roi[key][sub] = {}
        allois = list(dat.keys())
        for roi, tuff in dat.items():
            for roi2 in allois:
                if roi != roi2:
                    inter_roi[key][sub]['%s_%s'%(roi,roi2)] = {}
                    for i, stuff in tuff.items():
                        G = stuff['natural_nx']
                        Gmat = nx.to_numpy_matrix(G)
                        Gmat = stats.zscore(np.array(Gmat))
                        #############################
                        H = dat[roi2][i]['natural_nx']
                        Hmat = nx.to_numpy_matrix(H)
                        Hmat = stats.zscore(np.array(Hmat))
                        x = np.corrcoef(Gmat, Hmat)
                        z = int(np.divide(x.shape[0],2))
                        rows = np.array(list(range(z,x.shape[0],1)))
                        cols = np.array(list(range(0,z,1)))
                        y = stats.zscore(x[rows[:, None], cols])
                        inter_roi[key][sub]['%s_%s'%(roi,roi2)][i]=y

## Creating the multilayer graph

This takes forever, if going to do it again run in parallel

In [ ]:
network_outputs[100]['20002_neu_4']['top_n_by_degree']

In [ ]:
A = pool_outputs[100]['20002']['neu']['4']['network']
for edge in A.get_edges(data = True):
     print(edge)


In [ ]:
big_df['low']['20003'][5][big_df['low']['20003'][5]['source'] == '1154']

In [ ]:
df_dict['low']['20003']['Hippo'][5][df_dict['low']['20003']['Hippo'][5]['source'] == '1154']

In [ ]:
G = graphs['low']['20003']['Hippo'][5]['natural_nx']
G = nx.relabel_nodes(G, graphs['low']['20003']['Hippo'][5]['node_maps'])
df = nx.to_pandas_edgelist(G)
set(df['source'])

In [ ]:
df_dict = {}
for level, data in graphs.items():
    df_dict[level] = {}
    for sub, dat in data.items():
        dfkeeper = {}
        df_dict[level][sub] = {}
        for roi, da in dat.items():
            df_dict[level][sub][roi] = {}
            for i, graph in da.items():
                print(i)
                df_dict[level][sub][roi][i] = {}
                dfkeeper[i] = []
                G = graph['natural_nx']
                G = nx.relabel_nodes(G, graph['node_maps'])
                df = nx.to_pandas_edgelist(G)
                df['layer_target'] = '%s'%roi
                df['layer_source'] = '%s'%roi
                df = df.reindex(columns=['source','layer_source','target','layer_target','weight'])
                dfkeeper[i].append(df)
                for rois, num in inter_roi[level][sub].items():
                    if roi in rois:
                        mat = inter_roi[level][sub][rois][i]
                        ROIS = rois.split('_')
                        H = nx.from_numpy_matrix(mat)
                        H = nx.relabel_nodes(H, graph['node_maps'])
                        df2 = nx.to_pandas_edgelist(H)
                        df2['layer_target'] = '%s'%ROIS[0]
                        df2['layer_source'] = '%s'%ROIS[1]
                        df2 = df2.reindex(columns=['source','layer_source','target','layer_target','weight'])
                        dfkeeper[i].append(df2)
                df_dict[level][sub][roi][i] = pd.concat(dfkeeper[i])
#                 df_dict[level][sub][roi].to_csv(os.path.join(savepath,'%s_%s_%s_%s_DF.txt'%(level, sub, i, roi)), sep = '\t',header = None, index = False)
                
                
                

In [ ]:
df_dict = onetoughjar(os.path.join(savepath,'tmp','df_dict_sm_z*'))

In [ ]:
adillyofapickle(savepath,df_dict,'df_dict_sm_z')

In [6]:
savepath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/'

In [53]:
df_dict = onetoughjar(os.path.join(savepath,'tmp','df_dict_sm_z_*'))

['/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/df_dict_sm_z_06-23-2022_02-52-16', '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/df_dict_sm_z_06-30-2022_02-10-42', '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/df_dict_sm_z_06-23-2022_05-39-13', '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/df_dict_sm_z_06-23-2022_02-53-14']
/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/df_dict_sm_z_06-30-2022_02-10-42


This one also takes a while, could use parallel

In [54]:
df_dict['high']['20001']['Pu'].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [55]:
ROIS = ['Pu', 'Ca', 'NAC', 'EXA', 'dlPFC', 'Hippo', 'Ins']

In [60]:
big_boi = {}
for level, data in df_dict.items():
    big_boi[level] = {}
    for sub, dat in data.items():
        if sub == '20001' and level == 'high':
            print(sub)
        big_boi[level][sub] = {}
        try:
            for x in range(0,100,1):
                if sub == '20001'and level == 'high':
                    print(x)
                big_boi[level][sub][x] = []
                for roi, da in dat.items():
                    if sub == '20001'and level == 'high':
                        print(roi)
                    big_boi[level][sub][x].append(df_dict[level][sub][roi][x])
        except KeyError:
            print('non')

non
20001
0
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
1
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
2
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
3
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
4
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
5
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
6
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
7
Pu
Ca
NAC
EXA
dlPFC
Hippo
Ins
8
Pu
non
non
non
non
non
non
non
non
non
non
non
non
non
non
non
non
non


In [61]:
big_boi['high']['20001'][8]

[]

In [ ]:
for level, data in big_boi.items():
    print(level)
    for sub, dat in data.items():

In [12]:
def catty(dat):
    try:
        for i, da in dat.items():
            print(i)
            df = pd.concat(da).drop_duplicates()
            df = df[df['weight'] >= 3]
    except:
        print('oops')
    return({i:df})

In [13]:
big_df = {}
for level, data in big_boi.items():
    big_df[level] = {}
    for sub, dat in data.items():
        big_df[level][sub] = {}
        for i, da in dat.items():
#             big_df[level][sub][i] = {}
            print(i)
            try:
                df = pd.concat(da).drop_duplicates()
            except:
                print('non')
            df = df[df['weight'] >= 3]
            big_df[level][sub][i] = df
            
        

0
1
2
3
non
0
1
2
3
4
5
6
7
8
non
0
1
2
3
4
5
6
7
8
9
10
11
12
non
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
non
0
1
2
3
4
5
6
7
8
9
10
non
0
1
2
3
4
5
6
7
8
9
10
non
0
1
2
3
4
5
6
7
8
9
10
11
non
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
non
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
non
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
non
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
non
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
non
0
1
2
3
4
5
6
7
8
non
0
1
2
3
4
5
6
7
8
9
non
0
1
2
3
4
5
6
7
8
9
10
11
12
non
0
1
2
3
4
5
6
7
8
9
10
non
0
1
2
3
4
5
non
0
1
2
3
4
5
6
7
8
9
10
11
non


In [19]:
big_df['high'].keys()

dict_keys(['20008', '20001', '20002', '20003', '20004', '20010'])

In [17]:
big_df['high']['20001'].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [16]:
big_df['high']['20008'].keys()

dict_keys([0, 1, 2, 3])

In [18]:
big_df['high']['20003'].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])

In [23]:
big_df['high']['20002'][1]

,source,layer_source,target,layer_target,weight
12,480,Pu,492,Pu,3.414877
13,480,Pu,493,Pu,3.715415
27,480,Pu,507,Pu,7.922115
93,481,Pu,482,Pu,3.794113
94,481,Pu,483,Pu,7.767090
...,...,...,...,...,...
4247,564,Ins,569,Ins,6.330419
4253,565,Ins,568,Ins,4.355942
4254,565,Ins,569,Ins,5.886004
4258,566,Ins,567,Ins,6.408838


In [25]:
big_df['high']['20003'][2]

,source,layer_source,target,layer_target,weight
4,540,Pu,544,Pu,4.361277
5,540,Pu,545,Pu,5.816220
65,541,Pu,545,Pu,7.339853
123,542,Pu,544,Pu,3.229410
124,542,Pu,545,Pu,5.570586
...,...,...,...,...,...
1858,593,Ins,596,Ins,5.863273
1865,594,Ins,596,Ins,6.296087
1883,597,Ins,599,Ins,3.779028
1884,597,Ins,600,Ins,6.006777


In [47]:
big_df['high']['20001'].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [21]:
big_df['high']['20003'][1]

,source,layer_source,target,layer_target,weight
1,360,Pu,361,Pu,5.477226
35,361,Pu,365,Pu,4.096017
64,362,Pu,365,Pu,4.208055
91,363,Pu,364,Pu,3.849966
173,366,Pu,368,Pu,3.946520
...,...,...,...,...,...
370,375,Ins,385,Ins,3.317162
399,377,Ins,385,Ins,3.939923
461,383,Ins,384,Ins,4.112564
485,386,Ins,390,Ins,3.838569


In [ ]:
adillyofapickle(savepath,big_df,'big_df')

In [ ]:
big_df = onetoughjar(os.path.join(savepath,'tmp','big_df*'))

In [ ]:
for level, data in big_df.items():
    for sub, dat in data.items():
        for i, da in dat.items():
            da.to_csv(os.path.join(savepath,'%s_%s_%s.txt'%(level,sub,i)),sep = ' ', header = False, index = False)  

In [ ]:
def multitask(txt_path):
    level = txt_path.split('/')[-1].split('_')[0]
    sub = txt_path.split('/')[-1].split('_')[1]
    i = txt_path.split('/')[-1].split('_')[2].split('.')[0]
    net = multinet.multi_layer_network().load_network(txt_path, directed=False, input_type="multiedgelist")
    out = {'%s'%sub:{'%s'%level:{'%s'%i:{'network':net}}}}
    return(out)

In [ ]:


max_pool = 4
data = glob.glob(os.path.join(savepath,'*_*_*.txt'))

with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.imap(multitask,
                   data),
            total=len(data)
        )
    )    

#multi_high = dict(zip(subs, pool_outputs))

In [ ]:
adillyofapickle(savepath,pool_outputs,'multi_nets')

In [ ]:
def g_louvain_communities(network, output="mapping"):
#     from .community_louvain import *
    try:
        G = nx.Graph()
        for edge in network.core_network.edges():
            G.add_edge(edge[0], edge[1])
        network = G

    except Exception as es:
        pass ## nx input directly.

    partition = community_louvain.best_partition(network)
    MOD = community_louvain.modularity(partition, G)
    if output == "partition":
        dx_hc = defaultdict(list)
        for a, b in partition.items():
            dx_hc[b].append(a)
        return dx_hc
    return {'partition':partition,'Q':MOD}

In [ ]:
def doitall(pool_out):
    out = {}
    for sub, data in pool_out.items():
#         print(sub)
        for level, dat in data.items():
            for i, da in dat.items():
                net = da['network']
                try:
                    out['%s_%s_%s'%(sub,level,i)] = {'partition_dict':g_louvain_communities(net),
                                                    'top_n_by_degree': identify_n_hubs(net.core_network, 20),
                                                    'tangled': compute_entanglement_analysis(net)}
                except ValueError:
                    print('skip')
    return(out)


In [ ]:
max_pool = 4
data = pool_outputs

with Pool(max_pool) as p:
    network_outputs = list(
        tqdm(
            p.imap(doitall,
                   data),
            total=len(data)
        )
    )    



In [ ]:
network_outputs[20]['20008_low_8']

In [ ]:
adillyofapickle(savepath,network_outputs,'network_outputs')

In [7]:
savepath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/'

In [11]:
network_outputs = onetoughjar(os.path.join(savepath, 'tmp','network_outputs*'))

['/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/network_outputs_06-30-2022_06-11-24', '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/network_outputs_06-30-2022_03-11-07']
/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/tmp/network_outputs_06-30-2022_06-11-24


In [20]:
network_outputs[100]['20002_neu_4']['tangled']

[{'Entanglement intensity': 0.1004622065900731,
  'Layer entanglement': {'Pu': 0.2365100663505716,
   'Ca': 0.3807767494810215,
   'NAC': 0.6059834962501386,
   'EXA': 0.16776750775401666,
   'dlPFC': 0.23239481802984083,
   'Hippo': 0.3668321303878455,
   'Ins': 0.4638285867127768},
  'Entanglement homogeneity': 0.9275601016252504,
  'Normalized homogeneity': 0.7561957600425006}]

In [71]:
chunks = [network_outputs[x:x+100] for x in range(0, len(network_outputs), 100)]

In [74]:
len(chunks[2])

34

In [ ]:
def detangle(data, sub, level, i):
    df = pd.DataFrame.from_dict(data['tangled'][0])
    df['ROI'] = df.index
    df['sub'] = sub
    df['level'] = level
    df['i'] = i
    df.reset_index(drop=True, inplace=True)
    return(df)

In [ ]:
def tipytop(data, sub, level, i):
    df = pd.DataFrame.from_dict(data['top_n_by_degree'], orient = 'index')
    df['ROI'] = df.index
    df['sub'] = sub
    df['level'] = level
    df['i'] = i
    df.reset_index(drop=True, inplace=True)
    df = df.rename(columns={0: 'degree'})
    return(df)

In [ ]:
def Q(data, sub, level, i):
    data = data['partition_dict']['Q']
    out ={'sub': [sub], 'level':[level], 'i':[i], 'Q': [data]}
    df = pd.DataFrame.from_dict(out)
    return(df)

In [81]:
def makeDF(list_data):
    DF_entang = []
    DF_degree = []
    DF_Q = []
    for x in list_data:
        for label, data in x.items():
            base = label.split('_')
            sub = base[0]
            level = base[1]
            i = base[2]
            DF_entang.append(detangle(data, sub, level, i))
            DF_degree.append(tipytop(data, sub, level, i))
            DF_Q.append(Q(data, sub, level, i))
    return({'entanglement':DF_entang, 'degree':DF_degree, 'Q':DF_Q})

In [83]:
max_pool = len(chunks)
data = chunks

with Pool(max_pool) as p:
    list_DFS = list(
        tqdm(
            p.imap(makeDF,
                   chunks),
            total=len(chunks)
        )
    )    

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.31it/s]


In [102]:
entang = []
Q = []
degree = []
for x in list_DFS:
    entang.append(pd.concat(x['entanglement']))
    degree.append(pd.concat(x['degree']))
    Q.append(pd.concat(x['Q']))
total_entang = pd.concat(entang)
total_degree = pd.concat(degree)
total_Q = pd.concat(Q)

In [107]:
total_entang.to_csv(os.path.join(savepath,'total_entang.csv'), index=False)
total_degree.to_csv(os.path.join(savepath,'total_degree.csv'), index=False)
total_Q.to_csv(os.path.join(savepath,'total_Q.csv'), index=False)


In [139]:
test = total_entang[(total_entang['sub'] == '20001') & (total_entang['ROI'] == 'Ca')]






In [140]:
test['i'] = pd.to_numeric(test['i'])
test = test.sort_values('i')

In [141]:
test

,Entanglement intensity,Layer entanglement,Entanglement homogeneity,Normalized homogeneity,ROI,sub,level,i
0,0.295860,0.418865,0.967993,0.838496,Ca,20001,low,0
0,0.250677,0.446756,0.966881,0.835699,Ca,20001,high,0
0,0.303437,0.431606,0.965673,0.832713,Ca,20001,neu,0
0,0.184181,0.326681,0.959719,0.818693,Ca,20001,low,1
0,0.241069,0.420009,0.963645,0.827813,Ca,20001,high,1
0,0.196125,0.413892,0.967721,0.837808,Ca,20001,neu,1
0,0.198072,0.450767,0.961126,0.821908,Ca,20001,neu,2
0,0.366173,0.314038,0.900503,0.713602,Ca,20001,low,2
0,0.282346,0.460137,0.937239,0.773254,Ca,20001,high,2
0,0.190435,0.481235,0.929384,0.759323,Ca,20001,neu,3


In [128]:
ind = []
for i, value in enumerate(list(model_pattern['20001']['model'])):
    print(i)
    print(value)
    if value == 0:
        
    for index, row in test[].iterrows():
        print(row['c1'], row['c2'])
    

0
0.0
1
1.0
2
0.0
3
-1.0
4
1.0
5
0.0
6
-1.0
7
0.0
8
1.0
9
0.0
10
-1.0
11
0.0
12
1.0
13
-1.0
14
0.0
15
1.0
16
0.0
17
-1.0
18
0.0
19
1.0
20
-1.0
21
0.0
22
-1.0
23
0.0
24
-1.0
25
0.0
26
-1.0
27
0.0
28
1.0
29
0.0
30
1.0
31
0.0
